In [20]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn import model_selection


In [21]:
dataset = pd.read_csv('../CSVs/Dataset_practica_completo.csv')
dataset.columns

Index(['Temporada', 'Jornada', 'Dif PTOS', 'Dif POS', 'Dif FORMA',
       'Dif PTS L/V', 'Dif G AF', 'Dif G EC', 'Target'],
      dtype='object')

In [22]:
dataset.rename(columns={"Target": "Resultado"}, inplace=True)

In [23]:
X = dataset.iloc[:,2:-1]
y = dataset.iloc[:,-1]

In [24]:
X

,Dif PTOS,Dif POS,Dif FORMA,Dif PTS L/V,Dif G AF,Dif G EC
0,-0.600,10,0.0,0.000,-0.400,-0.400
1,0.200,-1,4.0,0.667,1.600,-0.200
2,0.400,-6,-1.0,0.400,1.000,-0.600
3,-0.300,4,-1.0,-0.134,-0.600,0.800
4,0.000,-1,3.0,0.267,0.600,-0.600
...,...,...,...,...,...,...
5455,0.372,-7,6.0,0.274,0.823,-0.118
5456,0.600,-4,1.0,0.314,0.823,-0.353
5457,-0.057,2,-4.0,0.176,0.176,-0.589
5458,-0.058,1,-1.0,0.000,0.177,-0.177


In [25]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

'''
Por defecto coge soft voting, si pueden todos los clasificadores.
En el caso del SVM tiene que realizar una validación cruzada ya que
no soporta el predict_proba()
'''

log_clf = LogisticRegression(random_state=42) # Mezcla los datos
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42) # Se usa en combinación con otro arg
svm_clf = SVC(gamma="scale", random_state=42)

estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)]

voting_clf = VotingClassifier(estimators=estimators, voting='hard')

In [27]:
voting_clf.fit(data_train, target_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [28]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(data_train, target_train)
    y_pred = clf.predict(data_test)
    print(clf.__class__.__name__, accuracy_score(target_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression 0.5228937728937729
RandomForestClassifier 0.4716117216117216
SVC 0.5045787545787546


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier 0.5137362637362637


# BAGGING CLASSIFIER

In [29]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(max_depth=3,random_state=42)

bag_clf = BaggingClassifier(
    base_estimator = estimator,
    n_estimators=500, # Cantidad de árboles
    max_samples=100, # Muestras utilizadas en boostrapping
    bootstrap=True, # Usamos boostrapping
    # max_features = 3 # Features que utiliza en el boostrapping. Cuanto más bajo, mejor generalizará y menos overfitting
    random_state=13)




In [30]:
name='Bagging Classifier'
kfold = model_selection.StratifiedKFold(n_splits=10) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(bag_clf, data_train, target_train, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)
print(msg)

[0.49885584 0.51258581 0.52173913 0.52402746 0.51258581 0.52173913
 0.48283753 0.52631579 0.50917431 0.5206422 ]
Bagging Classifier: 0.513050 (0.012786)


In [31]:
bag_clf.fit(data_train, target_train)
y_pred = bag_clf.predict(data_test)
accuracy_score(target_test, y_pred)

0.5238095238095238

In [48]:
prediccion_df = pd.DataFrame(y_pred, columns=['Prediccion'])
prediccion_df['Prediccion'].value_counts()

L    875
V    211
E      6
Name: Prediccion, dtype: int64

In [49]:
875/prediccion_df['Prediccion'].value_counts().sum()

0.8012820512820513

# MODELO MULTICLASS Y AÚN ASÍ DA % ALTOS DE LOCAL. 

In [33]:
y_pred = bag_clf.predict_proba(data_test)
y_pred

array([[0.25390707, 0.34557453, 0.4005184 ],
       [0.27278711, 0.51886081, 0.20835208],
       [0.31167869, 0.43757218, 0.25074913],
       ...,
       [0.24753533, 0.57472213, 0.17774254],
       [0.21679406, 0.63880871, 0.14439724],
       [0.24605295, 0.53688044, 0.21706661]])

In [34]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(max_depth=5,random_state=42)

bag_clf = BaggingClassifier(
    base_estimator = estimator,
    n_estimators=200, # Cantidad de árboles
    max_samples=150, # Muestras utilizadas en boostrapping
    bootstrap=True, # Usamos boostrapping
    # max_features = 3 # Features que utiliza en el boostrapping. Cuanto más bajo, mejor generalizará y menos overfitting
    random_state=12)


bag_clf.fit(data_train, target_train)
y_pred = bag_clf.predict(data_test)
accuracy_score(target_test, y_pred)

0.5146520146520146

In [35]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500,
                                 max_leaf_nodes=30,
                                 max_depth=3,
                                 min_samples_leaf=10,
                                 random_state=42)
#rnd_clf.fit(data_train, target_train)

#y_pred_rf = rnd_clf.predict(data_test)
# np.sum(y_test == y_pred_rf) / len(y_test) 
#accuracy_score(target_test, y_pred_rf)

In [36]:
name='Random Forest Classifier'
kfold = model_selection.StratifiedKFold(n_splits=10) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(rnd_clf, data_train, target_train, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)
print(msg)

[0.49656751 0.51258581 0.5194508  0.51029748 0.50572082 0.50800915
 0.49199085 0.53318078 0.49770642 0.51605505]
Random Forest Classifier: 0.509156 (0.011581)


In [37]:
rnd_clf.fit(data_train, target_train)

y_pred_rf = rnd_clf.predict(data_test)
# np.sum(y_test == y_pred_rf) / len(y_test) 
accuracy_score(target_test, y_pred_rf)

0.5091575091575091

# Adam Boost Classifier

In [38]:
from sklearn.ensemble import AdaBoostClassifier

estimator = DecisionTreeClassifier(max_depth=1)

ada_clf = AdaBoostClassifier(base_estimator = estimator,
                             n_estimators=200,
                             learning_rate=0.5,
                             random_state=42)


In [39]:
name='Adam Bost Classifier'
kfold = model_selection.StratifiedKFold(n_splits=10) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(ada_clf, data_train, target_train, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)
print(msg)

[0.48741419 0.52402746 0.52173913 0.51716247 0.50343249 0.52402746
 0.48970252 0.51716247 0.48394495 0.50917431]
Adam Bost Classifier: 0.507779 (0.014931)


In [40]:
ada_clf.fit(data_train, target_train)

y_pred = ada_clf.predict(data_test)
accuracy_score(target_test, y_pred)

0.5137362637362637

# GradientBoostingClassifier

In [41]:
from sklearn.ensemble import GradientBoostingClassifier

gbct = GradientBoostingClassifier(max_depth=5,
                                 n_estimators=200,
                                 learning_rate=0.2,
                                 random_state=42)
gbct.fit(data_train, target_train)


y_pred_gbct = gbct.predict(data_test)
accuracy_score(target_test, y_pred_gbct)

0.4706959706959707

In [42]:
name='GradientBoostingClassifier'
kfold = model_selection.StratifiedKFold(n_splits=10) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(gbct, data_train, target_train, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)
print(msg)

KeyboardInterrupt: 